In [6]:
import yaml
import shutil
import neorl2
import random
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from collections import OrderedDict

In [7]:
env = neorl2.make("Pipeline")
data,_ = env.get_dataset()

In [8]:
Q_out = data["obs"][:,0:1]
Q_target = data["obs"][:,1:2]
CV_in = data["action"]

index =np.where(np.logical_or(data["done"], data["truncated"]))[0]+1
index = [0,] + list(index)
id = 1
for start_index,end_index in zip(index[:-1],index[1:]):
    Q_out = data["obs"][start_index:end_index,0:1]
    Q_target = data["obs"][start_index:end_index,1:2]
    CV_in = data["action"][start_index:end_index]
    history = data["obs"][start_index:end_index,2:]
    
    
    columns = ["Q_out","Q_target","CV_in"]
    columns += [f"Q_out_history_{i}" for i in range(25)]
    columns += [f"CV_in_history_{i}" for i in range(25)]
    df = pd.DataFrame(np.concatenate([Q_out, Q_target,CV_in,history],axis=1),columns=columns)
    
    start_date = datetime(2010, 3, 1) 
    end_date = datetime(2022, 3, 31) 
    start_time = datetime.combine(random.choice(pd.date_range(start=start_date, end=end_date)).date(), datetime.min.time())
    
    
    df['time'] = [start_time + timedelta(seconds=1)*i for i in range(len(df))]
    timestamp_col = df.pop('time')
    df.insert(0, 'time', timestamp_col)
    
    df.to_csv(f"./csv/{id}.csv")
    id += 1

In [14]:
config_dict = {}
config_dict['metadata'] = dict()

graph ={
    "CV_in": ["Q_out", "Q_target","Q_out_history", "CV_in_history"],
    "next_Q_out": ["Q_out",  "CV_in","Q_out_history", "CV_in_history"],
    "next_Q_out_history" : ["Q_out_history", "Q_out"],
    "next_CV_in_history" : ["CV_in_history", "CV_in"]
}

config_dict['metadata']['graph'] = graph

nodes = ["Q_out","Q_target","CV_in","Q_out_history", "CV_in_history"]
config_dict['metadata']['columns'] = []

for node in nodes:
    if node == "Q_out":
        for column in ["Q_out",]:
            config_dict['metadata']['columns'].append({
                f'{column}' : {
                    'type' : 'continuous',
                    'dim' : node
                }
            })
    elif node == "Q_target":
        for column in ["Q_target",]:
            config_dict['metadata']['columns'].append({
                f'{column}' : {
                    'type' : 'continuous',
                    'dim' : node
                }
            })
    elif node == "Q_out_history":
        for column in [f"Q_out_history_{i}" for i in range(25)]:
            config_dict['metadata']['columns'].append({
                f'{column}' : {
                    'type' : 'continuous',
                    'dim' : node
                }
            })
    elif node == "CV_in_history":
        for column in [f"CV_in_history_{i}" for i in range(25)]:
            config_dict['metadata']['columns'].append({
                f'{column}' : {
                    'type' : 'continuous',
                    'dim' : node
                }
            })
    elif node == "CV_in":
        for column in ["CV_in",]:
            config_dict['metadata']['columns'].append({
                f'{column}' : {
                    'type' : 'continuous',
                    'dim' : node
                }
            })
    else:
        raise NotImplementedError
    
expert_functions = {}

expert_functions["next_Q_out_history"] = {
        "node_function":"expert_functions.next_Q_out_history"
        }

expert_functions["next_CV_in_history"] = {
        "node_function":"expert_functions.next_CV_in_history"
        }

shutil.copy2("./expert_functions.py",  "../revive_train/data/expert_functions.py")
    
if expert_functions:
    config_dict['metadata']['expert_functions'] = expert_functions
        
with open("../revive_train/data/pipeline.yaml", 'w') as f:
    yaml.dump(dict(config_dict), f)
